In [2]:
# import libraries
import geopandas as gpd
import momepy
import networkx as nx
import shapely

In [3]:
### PREPROCESSING EXPECTED FROM USER!

# read in toy graph (Bubenec)
gdf = gpd.read_file(momepy.datasets.get_path("bubenec"), layer="streets")

# remove false nodes (interstitital nodes of degree 2)
gdf = momepy.remove_false_nodes(gdf)

# make primal graph
graph = momepy.gdf_to_nx(
    gdf, 
    preserve_index=True, # !! preserving index needed for unambiguous mapping to coins!
    approach="primal"
)

# get momempy lines of graph
lines = momepy.nx_to_gdf(
    graph,
    points=False,
    lines=True
)

# get COINS of graph lines
coins = momepy.COINS(
    lines,
    angle_threshold=0,
    flow_mode=False
) # this will be the input to strokes_to_graph

/var/folders/66/3jkth_7d5gggg6pyr8yywwt40000gn/T/ipykernel_88547/491392270.py:7: FutureWarning: `remove_false_nodes` is deprecated and will be removed in momepy 1.1. The function has been improved and moved to the `neatnet` package and can be used as `neatnet.remove_interstitial_nodes`. See https://uscuni.org/neatnet for details.
  gdf = momepy.remove_false_nodes(gdf)


In [8]:
### USING THE FUNCTIONS

# make stroke graph
stroke_graph = momepy.strokegraph.coins_to_nx(coins)

# add access (and with it, degree and connectivity)
stroke_graph = momepy.strokegraph.compute_stroke_access(stroke_graph)

# add orthogonality
stroke_graph = momepy.strokegraph.compute_stroke_orthogonality(stroke_graph)

# add spacing
stroke_graph = momepy.strokegraph.compute_stroke_spacing(stroke_graph)


In [14]:
stroke_graph.nodes[0]

{'edge_indeces': [0, 3, 15, 27],
 'geometry': <POINT (1603374.663 6464077.898)>,
 'stroke_geometry': <LINESTRING (1603278.899 6463669.186, 1603283.731 6463690.028, 1603314.444 6...>,
 'stroke_length': 839.5666838320316,
 'x': 1603374.6625343116,
 'y': 6464077.898491419,
 'connectivity': 0,
 'stroke_connectivity': 8,
 'stroke_degree': 5,
 'stroke_access': 3,
 'stroke_orthogonality': np.float64(68.74678997354196),
 'stroke_spacing': 104.94583547900395}

In [ ]:
# TODO: VISUALIZE (1) coins-to-nx logic; (2) stroke metrics